# べき級数の収束半径から円周率を求める

べき級数

$$\sum_{n = 0}^{\infty} a_n z^n \quad (a_n \in \mathbb{C})$$

には、収束半径と呼ばれる数 $r \in [0, \infty]$ が一意的に定まり、$\sum_{n = 0}^{\infty} a_n z^n$ は $|z| < r$ を満たすとき収束し、$|z| > r$ を満たすとき発散します。

収束半径は、$\lim_{n\to\infty}\left|\frac{a_n}{a_{n+1}}\right|$ が ($\infty$ の場合も含めて) 存在すれば

$$r = \lim_{n\to\infty}\left|\frac{a_n}{a_{n+1}}\right|$$

が成り立ちます。これをダランベールの収束判定法と言います。上記の極限が存在しない場合でも

$$\begin{align}
r = \liminf_{n \to \infty}\frac{1}{\sqrt[n]{|a_n|}}
\end{align}$$

が成り立ちます。これをコーシー・アダマールの収束判定法と言います。このように、係数のある極限を取ると収束半径を求めることができます。

一方、係数が分からなくても収束半径が分かる場合があります。$\mathbb{C}$ の原点を含む領域 $D$ 上で関数 $f(z)$ が正則で、$D$ に含まれる開円板 $|z -a| < r$ 上で $f(z) \neq 0$ を満たし、$|z -a| = r$ を満たすある $z_0$ において $f(z_0) = 0$ を満たすとき、$\frac{1}{f(z)}$ は $a$ を中心としてべき級数展開可能で、

$$\frac{1}{f(z)} = \sum_{n=0}^{\infty} a_n (z-a)^n$$

の収束半径は $r$ となります。よって $r$ が $\pi$ と関わる値になるように $f(z)$ をとり、そのべき級数展開の係数から $r$ を計算することで、円周率を求められる場合があります。

## $\frac{1}{\cos z}$ から円周率を求める (セカント数)

$\cos z = 0$ の解は $z = \frac{\pi}{2} + n\pi$ $(n \in \mathbb{Z})$ であり、べき級数展開

$$\begin{align}
\frac{1}{\cos z} = \sum_{n=0}^{\infty} a_n z^n
\end{align}$$

の収束半径は $\frac{\pi}{2}$ となります。

$$\begin{align}
1 = \cos z \cdot \frac{1}{\cos z} = \left(\sum_{n=0}^{\infty} \frac{(-1)^n}{(2n)!} z^{2n} \right) \left(\sum_{n=0}^{\infty} a_n z^n\right) 
\end{align}$$

の両辺の次数を比較すると

$$\begin{gather}
a_0 = 1, \quad a_1 = 0 \\
\sum_{k=0}^{n} (-1)^{n-k} \frac{1}{(2n-2k)!} a_{2k} = 0 \\
\sum_{k=0}^{n} (-1)^{n-k} \frac{1}{(2n-2k)!} a_{2k+1} = 0 
\end{gather}$$

という式が得られます。これから $a_{2n+1} = 0$,

$$a_{2n} = -\sum_{k=0}^{n-1} (-1)^{n-k} \frac{1}{(2n-2k)!} a_{2k}$$

により $a_n$ を計算することができます。

$S_n = n! a_n$ はセカント数と呼ばれており、

$$\begin{gather}
\frac{1}{\cos z} = \sum_{n=0}^{\infty} \frac{S_n}{n!} z^n, \\
S_{2n} = -\sum_{k=0}^{n-1} (-1)^{n-k} \binom{2n}{2k} S_{2k} \\
S_{2n+1} = 0
\end{gather}$$

が成り立ちます。セカント数は自然数なので手計算では計算しやすいですが、$n$ に関して急激に大きくなるので計算機での計算には向きません。

$w = z^2$ とおくと、$a_{2n+1} = 0$ から

$$\cos z = \sum_{n=0}^{\infty} a_{2n} w^n$$

となりますが、$|z| < \frac{\pi}{2}$ で収束することから $\sqrt{|w|} < \frac{\pi}{2}$ つまり $|w| < \frac{\pi^2}{4}$ で収束します。
よってダランベールの判定法から

$$\frac{\pi^2}{4} = \lim_{n \to \infty}\frac{|a_{2n}|}{|a_{2n+2}|} = \lim_{n \to \infty} \frac{\frac{|S_{2n}|}{(2n)!}}{\frac{|S_{2n+2}|}{(2n+2)!}} = \lim_{n \to \infty} \frac{|S_{2n}|}{|S_{2n+2}|}(2n+2)(2n+1),$$

コーシー・アダマールの判定法から

$$\frac{\pi}{2} = \liminf_{n \to \infty}\frac{1}{\sqrt[2n]{|a_{2n}|}} = \liminf_{n \to \infty} \frac{1}{\sqrt[2n]{\frac{|S_{2n}|}{(2n)!}}} = \liminf_{n \to \infty} \frac{\sqrt[2n]{(2n)!}}{\sqrt[2n]{|S_{2n}|}}$$

により円周率が求められると考えられます。ただし極限の存在は仮定します。

#### セカント数の計算

それではまず、セカント数を計算してみましょう。

In [1]:
from functools import lru_cache
import math
import numpy
from fractions import Fraction


class Coefficient():
    NAME = ""
    
    def print_values(self, m: int):
        for n in range(m+1):
            print(f"{self.NAME}_{n}: {self.frac(n)}")

    def print_exp_type(self, m: int):
        for n in range(m+1):
            print(f"{self.NAME.upper()}_{n}: {self.exp_type(n)}")

    def value(self, n: int) -> float:
        return float(self.frac(n))

    def frac(self, n: int) -> Fraction:
        raise NotImplementedError()

    def exp_type(self, n: int) -> Fraction:
        return self.frac(n) * math.factorial(n)


class SecantCoeff(Coefficient):
    NAME = "s"

    @lru_cache(maxsize=1000)
    def frac(self, n: int) -> Fraction:
        if n < 0:
            raise ValueError(f"Input: n = {n}. Input value must be > 0.")
        if n == 0:
            return Fraction(1,1)
        else:
            return -sum([
                (-1) ** (n-k) * self.frac(k) * Fraction(1, math.factorial(2*n - 2*k))
                for k in range(0, n)
            ])

    def exp_type(self, n: int) -> Fraction:
        return self.frac(n) * math.factorial(2*n)
        
sc = SecantCoeff()
print("係数")
sc.print_values(10)

print("\nセカント数")
sc.print_exp_type(10)

係数
s_0: 1
s_1: 1/2
s_2: 5/24
s_3: 61/720
s_4: 277/8064
s_5: 50521/3628800
s_6: 540553/95800320
s_7: 199360981/87178291200
s_8: 3878302429/4184557977600
s_9: 2404879675441/6402373705728000
s_10: 14814847529501/97316080327065600

セカント数
S_0: 1
S_1: 1
S_2: 5
S_3: 61
S_4: 1385
S_5: 50521
S_6: 2702765
S_7: 199360981
S_8: 19391512145
S_9: 2404879675441
S_10: 370371188237525


#### 円周率の計算 (セカント数)

それではセカント数から円周率を求めます。

In [2]:
import sys
sys.path.append('/home/jovyan/work/')
from lib.utils import print_pi, PI_50
DPS = 50

class Caluculator():
    def dAlem(self, n: int) -> float:
        raise NotImplementedError()

    def ch(self, n: int) -> float:
        raise NotImplementedError()

    def print_dAlem(self, m: int, interval: int = 1):
        I = range(1, m+1, interval)
        print("d'Alembert")
        for n in I:
            print_pi(
                self.dAlem(n), 
                PI_50, 
                DPS, 
                _format=f"n = {n}: "+"{pi} ({mdigit} 桁まで一致)"
            )
        
    def print_ch(self, m: int, interval: int = 1):
        I = range(1, m+1, interval)
        print("Cauchy-Hadamard")
        for n in I:
            print_pi(
                self.ch(n), 
                PI_50, 
                DPS, 
                _format=f"n = {n}: "+"{pi} ({mdigit} 桁まで一致)"
            )
    

class SecantPiCalculator(Caluculator):
    
    def __init__(self):
        self.coeff: Coefficient = SecantCoeff()
        
    def dAlem(self, n: int) -> float:
        val = abs(self.coeff.value(n)) / abs(self.coeff.value(n +1))
        return math.sqrt(val) * 2
    
    def ch(self, n: int) -> float:
        N = 2 * n
        val = numpy.math.pow(
            1 / abs(self.coeff.value(n)),
            1 / N
        )
        return val * 2

calc = SecantPiCalculator()
calc.print_dAlem(10)
print("")
calc.print_ch(10)

print("")
calc.print_ch(502, 50)

d'Alembert
n = 1: 3.0983866769659336 (0 桁まで一致)
n = 2: 3.1362502409359 (1 桁まで一致)
n = 3: 3.140971819535975 (2 桁まで一致)
n = 4: 3.141522452287124 (4 桁まで一致)
n = 5: 3.141584801776917 (4 桁まで一致)
n = 6: 3.141591779037649 (5 桁まで一致)
n = 7: 3.14159255633089 (6 桁まで一致)
n = 8: 3.141592642779764 (7 桁まで一致)
n = 9: 3.141592652388539 (8 桁まで一致)
n = 10: 3.141592653456315 (9 桁まで一致)

Cauchy-Hadamard
n = 1: 2.8284271247461903 (0 桁まで一致)
n = 2: 2.960331217969141 (0 桁まで一致)
n = 3: 3.0178462212865687 (0 桁まで一致)
n = 4: 3.048167578315654 (0 桁まで一致)
n = 5: 3.066613937134974 (0 桁まで一致)
n = 6: 3.0789836695482675 (0 桁まで一致)
n = 7: 3.0878507089562515 (0 桁まで一致)
n = 8: 3.094517838085012 (0 桁まで一致)
n = 9: 3.0997133431722568 (0 桁まで一致)
n = 10: 3.1038760280008484 (1 桁まで一致)

Cauchy-Hadamard
n = 1: 2.8284271247461903 (0 桁まで一致)
n = 51: 3.134161288396892 (1 桁まで一致)
n = 101: 3.137837982337834 (1 桁まで一致)
n = 151: 3.1390807537634693 (1 桁まで一致)
n = 201: 3.13970541674064 (1 桁まで一致)
n = 251: 3.140081269888733 (2 桁まで一致)
n = 301: 3.1403322799353965 (

セカント数だと、どちらの判定法もおそらく下から収束する形になります。ダランベールの判定法は速く収束しますが、コーシー・アダマールの判定法はかなり収束が遅いです。

## $\frac{1}{e^z + 1}$ から円周率を求める

$e^z + 1 = 0$ の解は $z = \pi i + 2\pi i n$ $(n \in \mathbb{Z})$ なので、$\frac{1}{e^z + 1}$ の $0$ を中心としたべき級数展開

$$\frac{1}{e^z + 1} = \sum_{n=0}^{\infty} d_n z^n$$

の収束半径は $\pi$ となります。

$$1 = (e^z +1) \cdot \left(\frac{1}{e^z +1} \right) =\left(2 +\sum_{n=1}^{\infty}\frac{1}{n!} z^n\right) \left(\sum_{n=0}^{\infty}d_n z^n\right)$$

から

$$d_0 = \frac{1}{2}, \quad 2 d_n + \sum_{k=0}^{n-1} \frac{d_k}{(n-k)!} = 0 $$ 

となります。よって

$$d_n = -\frac{1}{2}\sum_{k=0}^{n-1} \frac{d_k}{(n-k)!}$$

であり、$D_n = n! d_n$ とおくと

$$D_n = -\frac{1}{2}\sum_{k=0}^{n-1} n!\frac{D_k}{k!(n-k)!} = -\frac{1}{2}\sum_{k=0}^{n-1} \binom{n}{k} D_k$$

となります。

あまり自明ではありませんが、$D_n$ は $n$ が $2$ 以上の偶数の場合に $0$ になります。これは $\frac{1}{e^{z} +1}-\frac{1}{2}$ が奇関数であること、つまり

$$\begin{align}
\frac{1}{e^{-z} +1} -\frac{1}{2} &= \frac{1}{(e^{z} +1)e^{-z}} -\frac{1}{2} = \frac{e^{z}}{e^{z} +1} -\frac{1}{2} \\
&= 1 -\frac{1}{e^{z} +1} -\frac{1}{2} = -\left(\frac{1}{e^{z} +1} -\frac{1}{2}\right) \\
\end{align}$$

と、$z$ を $-z$ に置き換えると符号が逆転することからわかります。

ダランベールの判定法から、収束すれば

$$\begin{align}
\pi^2 = \lim_{n \to \infty} \frac{|d_{2n-1}|}{|d_{2n+1}|} = \lim_{n \to \infty} \frac{\frac{|D_{2n-1}|}{(2n-1)!}}{\frac{|D_{2n+1}|}{(2n+1)!}} = \lim_{n \to \infty} \frac{|D_{2n-1}|}{|D_{2n+1}|}2n(2n+1)
\end{align}$$

となり、コーシー・アダマールの判定法から

$$\pi = \liminf_{n \to \infty}\frac{1}{\sqrt[2n+1]{|d_{2n+1}|}} = \liminf_{n \to \infty} \frac{1}{\sqrt[2n+1]{\frac{|D_{2n+1}|}{(2n+1)!}}} = \liminf_{n \to \infty} \frac{\sqrt[2n+1]{(2n+1)!}}{\sqrt[2n+1]{|D_{2n+1}|}}$$

により円周率が求められると考えられます。

#### $D_n$ の計算

まずは $D_n$ の計算をしましょう。

In [3]:
from functools import lru_cache
import math
import numpy
from fractions import Fraction


class DnCoeff(Coefficient):
    NAME = "d"

    @lru_cache(maxsize=1000)
    def frac(self, n: int) -> Fraction:
        if n < 0:
            raise ValueError(f"Input: n = {n}. Input value must be > 0.")
        if n == 0:
            return Fraction(1, 2)
        else:
            return -sum([
                self.frac(k) / math.factorial(n-k)
                for k in range(0, n)
            ]) / 2

dc = DnCoeff()
print("係数")
dc.print_values(10)
print("\nn!倍")
dc.print_exp_type(10)

係数
d_0: 1/2
d_1: -1/4
d_2: 0
d_3: 1/48
d_4: 0
d_5: -1/480
d_6: 0
d_7: 17/80640
d_8: 0
d_9: -31/1451520
d_10: 0

n!倍
D_0: 1/2
D_1: -1/4
D_2: 0
D_3: 1/8
D_4: 0
D_5: -1/4
D_6: 0
D_7: 17/16
D_8: 0
D_9: -31/4
D_10: 0


#### 円周率の計算 ($D_n$)

それでは $D_n$ から円周率を求めます。

In [5]:
class DnPiCalculator(Caluculator):
    
    def __init__(self):
        self.coeff: Coefficient = DnCoeff()
        
    def dAlem(self, n: int) -> float:
        val = abs(self.coeff.value(2 * n -1)) / abs(self.coeff.value(2 * n +1))
        return math.sqrt(val)
    
    def ch(self, n: int) -> float:
        N = 2 * n + 1
        val = numpy.math.pow(
            1 / abs(self.coeff.value(N)),
            1 / N
        )
        return val


calc = DnPiCalculator()
calc.print_dAlem(10)
print("")
calc.print_ch(10)
print("")
calc.print_ch(120, 10)

d'Alembert
n = 1: 3.4641016151377544 (0 桁まで一致)
n = 2: 3.1622776601683795 (1 桁まで一致)
n = 3: 3.1436209919735028 (2 桁まで一致)
n = 4: 3.1418096285318566 (3 桁まで一致)
n = 5: 3.1416164596811473 (3 桁まで一致)
n = 6: 3.141595287197927 (5 桁まで一致)
n = 7: 3.1415929457631706 (6 桁まで一致)
n = 8: 3.141592686035722 (7 桁まで一致)
n = 9: 3.1415926571941895 (8 桁まで一致)
n = 10: 3.1415926539902532 (9 桁まで一致)

Cauchy-Hadamard
n = 1: 3.634241185664279 (0 桁まで一致)
n = 2: 3.4375438551749578 (0 桁まで一致)
n = 3: 3.35086839347773 (0 桁まで一致)
n = 4: 3.303240143816447 (0 桁まで一致)
n = 5: 3.2732477496125556 (0 桁まで一致)
n = 6: 3.2526400866486145 (0 桁まで一致)
n = 7: 3.2376099828529266 (0 桁まで一致)
n = 8: 3.226163222118216 (0 桁まで一致)
n = 9: 3.217154897074284 (0 桁まで一致)
n = 10: 3.2098808696510237 (0 桁まで一致)

Cauchy-Hadamard
n = 1: 3.634241185664279 (0 桁まで一致)
n = 11: 3.2038842970236727 (0 桁まで一致)
n = 21: 3.174759270731871 (1 桁まで一致)
n = 31: 3.1641924257547167 (1 桁まで一致)
n = 41: 3.158731873926962 (1 桁まで一致)
n = 51: 3.155396570453738 (1 桁まで一致)
n = 61: 3.15314790879005

Dnだと、どちらの判定法もおそらく上から収束する形になります。セカント数と合わせると両側から円周率を評価できる可能性があります。セカント数と同様にダランベールの判定法は速く収束しますが、コーシー・アダマールの判定法はかなり収束が遅いです。

## $\frac{z}{e^z -1}$ 円周率を求める (ベルヌーイ数)

$e^z -1 = 0$ の解は $z = 2\pi i n \ (n \in \mathbb{Z})$ であり、

$$\frac{e^z -1}{z} = \sum_{n=0}^{\infty} \frac{1}{(n+1)!}z^n$$

が $z = 0$ で $1$ であることから、$\frac{z}{e^z -1}$ の極は $z = 2n\pi i \ (n \in \mathbb{Z} \setminus \{0\})$ となります。よって $\frac{z}{e^z -1}$ のべき級数展開

$$\frac{z}{e^z -1} = \sum_{n=0}^{\infty} b_n z^n$$

の収束半径は $2\pi$ となります。

$$\left(\sum_{n=0}^{\infty} b_n z^n\right) \left(\sum_{n=0}^{\infty} \frac{1}{(n+1)!}z^n\right) = 1$$

から $b_n$ を求めると

$$b_0 = 1, \quad \sum_{k=0}^n \frac{b_k}{(n+1-k)!} = 0$$

なので、

$$b_n = -\sum_{k=0}^{n-1} \frac{b_k}{(n+1-k)!}$$

となります。ちなみに $B_n = n! b_n$ はベルヌーイ数と呼ばれていて

$$B_n = -\sum_{k=0}^{n-1} \frac{n!}{(n+1-k)!} \frac{B_k}{k!} = -\frac{1}{n+1}\sum_{k=0}^{n-1} \binom{n+1}{k} B_k$$

を満たします。実は $B_n$ は $n$ が $3$ 以上の奇数のとき $B_n=0$ となり、偶数の場合は $B_n \neq 0$ となることが知られています。

べき級数の収束半径が $2 \pi$ であることから

$$\begin{gather}
\lim_{n\to\infty}\frac{|b_{2n}|}{|b_{2n+2}|} = \lim_{n\to\infty}(2n+2)(2n+1)\frac{|B_{2n}|}{|B_{2n+2}|} = 4 {\pi}^2, \\
\liminf_{n\to\infty} \frac{1}{\sqrt[2n]{|b_{2n}|}} = \liminf_{n\to\infty} \sqrt[2n]{\frac{(2n)!}{|B_{2n}|}} = 2 \pi
\end{gather}$$

が成り立ちます。

それでは実際に求めてみましょう。

In [7]:
from functools import lru_cache
import math
import numpy
from fractions import Fraction


class BernuolliCoeff(Coefficient):
    NAME = "b"
    
    @lru_cache(maxsize=1000)
    def frac(self, n: int) -> Fraction:
        if n < 0:
            raise ValueError(f"Input: n = {n}. Input value must be > 0.")
        if n == 0:
            return Fraction(1, 1)
        else:
            return -sum([
                self.frac(k) / math.factorial(n + 1 - k)
                for k in range(0, n)
            ])

bc = BernuolliCoeff()
print("係数")
bc.print_values(10)
print("\nベルヌーイ数")
bc.print_exp_type(10)

係数
b_0: 1
b_1: -1/2
b_2: 1/12
b_3: 0
b_4: -1/720
b_5: 0
b_6: 1/30240
b_7: 0
b_8: -1/1209600
b_9: 0
b_10: 1/47900160

ベルヌーイ数
B_0: 1
B_1: -1/2
B_2: 1/6
B_3: 0
B_4: -1/30
B_5: 0
B_6: 1/42
B_7: 0
B_8: -1/30
B_9: 0
B_10: 5/66


In [11]:
class BnPiCalculator(Caluculator):
    
    def __init__(self):
        self.coeff: Coefficient = BernuolliCoeff()
        
    def dAlem(self, n: int) -> float:
        val = abs(self.coeff.value(2 * n)) / abs(self.coeff.value(2 * n + 2))
        return math.sqrt(val) / 2
    
    def ch(self, n: int) -> float:
        N = 2 * n
        val = numpy.math.pow(
            1/ abs(self.coeff.value(N)),
            1 / N
        )
        return val / 2


calc = BnPiCalculator()
calc.print_dAlem(10)
print("")
calc.print_ch(10)
print("")
calc.print_ch(150, 10)

d'Alembert
n = 1: 3.8729833462074166 (0 桁まで一致)
n = 2: 3.24037034920393 (0 桁まで一致)
n = 3: 3.1622776601683795 (1 桁まで一致)
n = 4: 3.146426544510455 (2 桁まで一致)
n = 5: 3.1427678676214668 (2 桁まで一致)
n = 6: 3.141882965902545 (3 桁まで一致)
n = 7: 3.1416648546853394 (3 桁まで一致)
n = 8: 3.1416106625961344 (3 桁まで一致)
n = 9: 3.1415971512924634 (5 桁まで一致)
n = 10: 3.141593777512209 (5 桁まで一致)

Cauchy-Hadamard
n = 1: 1.7320508075688772 (0 桁まで一致)
n = 2: 2.5900200641113513 (0 桁まで一致)
n = 3: 2.7908316238670965 (0 桁まで一致)
n = 4: 2.87938824021137 (0 桁まで一致)
n = 5: 2.9309182203146773 (0 桁まで一致)
n = 6: 2.965207805342988 (0 桁まで一致)
n = 7: 2.9898254214391717 (0 桁まで一致)
n = 8: 3.0083965582731698 (0 桁まで一致)
n = 9: 3.022914696021679 (0 桁まで一致)
n = 10: 3.034578330042229 (0 桁まで一致)

Cauchy-Hadamard
n = 1: 1.7320508075688772 (0 桁まで一致)
n = 11: 3.0441544716562277 (0 桁まで一致)
n = 21: 3.090170852802982 (0 桁まで一致)
n = 31: 3.1066658980313635 (1 桁まで一致)
n = 41: 3.1151486488948947 (1 桁まで一致)
n = 51: 3.1203161451311514 (1 桁まで一致)
n = 61: 3.1237941964255

## $\frac{1}{\sin z +1}$ から円周率を求める

ここまでは全て、分母が $e^iz + a$ という形でしたが、少し違う形式でどうなるか確認してみましょう。$\sin z +1$ は

$$\begin{align}
\sin z + 1 = \frac{e^{iz} -e^{-iz}}{2i} + 1 = \frac{1}{2i e^{iz}}(e^{2iz} +2i e^{iz} -1) = \frac{1}{2i e^{iz}}(e^{iz} +i)^2 \\
\end{align}$$

なので、$\frac{1}{\sin z +1}$ の分母は $(e^{iz} +i)^2$ という形をしており、$\sin z +1$ の解は $z = -\frac{\pi}{2} + 2n\pi$ $(n \in \mathbb{Z})$ となります。よって $\sin z +1$ の $0$ を中心とするべき級数展開

$$\begin{align}
\frac{1}{\sin z+1} = \sum_{n=0}^{\infty} c_n z^n
\end{align}$$

の収束半径 $\frac{\pi}{2}$ となります。

$$\begin{align}
1 = (\sin z +1) \cdot \frac{1}{\sin z+1} = \left(1 + \sum_{n=0}^{\infty}\frac{(-1)^n}{(2n+1)!} z^{2n+1} \right) \left(\sum_{n=0}^{\infty} c_n z^n\right)
\end{align}$$

から $c_n$ を求めると、

$$\begin{align}
c_0 &= 1, \\
c_{2n+1} &= -\sum_{k=0}^n \frac{(-1)^{n-k}}{(2n-2k+1)!} c_{2k} \quad (n \geq 0),\\
c_{2n} &= -\sum_{k=1}^{n} \frac{(-1)^{n-k}}{(2n-2k+1)!} c_{2k-1} \quad (n \geq 1)
\end{align}$$

となります。$C_n = n! c_n$ とおくと

$$\begin{align}
C_0 &= 1, \\
C_{2n+1} &= -\sum_{k=0}^n (-1)^{n-k} \binom{2n+1}{2k} C_{2k} \quad (n \geq 0),\\
C_{2n} &= -\sum_{k=1}^{n} (-1)^{n-k} \binom{2n}{2k-1} C_{2k-1} \quad (n \geq 1)\\
\end{align}$$

により $C_n$ を求めることができます。


In [12]:
from functools import lru_cache
import math
import numpy
from fractions import Fraction


class CnCoeff(Coefficient):
    NAME = "c"
    
    @lru_cache(maxsize=1000)
    def frac(self, n: int) -> Fraction:
        if n < 0:
            raise ValueError(f"Input: n = {n}. Input value must be > 0.")
        if n == 0:
            return Fraction(1, 1)
        else:
            if n % 2 == 0:
                N = n // 2
                return -sum([
                    (-1) ** (N - k) * self.frac(2 * k -1) / math.factorial(2 * N - 2 * k + 1)
                    for k in range(1, N+1)
                ])
            else:
                N = (n - 1) // 2
                return -sum([
                    (-1) ** (N - k) * self.frac(2 * k) / math.factorial(2 * N - 2 * k + 1)
                    for k in range(0, N+1)
                ])

cc = CnCoeff()
print("係数")
cc.print_values(10)
print("\nn!倍")
cc.print_exp_type(10)

係数
c_0: 1
c_1: -1
c_2: 1
c_3: -5/6
c_4: 2/3
c_5: -61/120
c_6: 17/45
c_7: -277/1008
c_8: 62/315
c_9: -50521/362880
c_10: 1382/14175

n!倍
C_0: 1
C_1: -1
C_2: 2
C_3: -5
C_4: 16
C_5: -61
C_6: 272
C_7: -1385
C_8: 7936
C_9: -50521
C_10: 353792


In [13]:
class CnPiCalculator(Caluculator):
    
    def __init__(self):
        self.coeff: Coefficient = CnCoeff()
        
    def dAlem(self, n: int) -> float:
        val = abs(self.coeff.value(n)) / abs(self.coeff.value(n+1))
        return val * 2
    
    def ch(self, n: int) -> float:
        val = numpy.math.exp(
            1 / self.coeff.value(n),
            1 / n
        )
        return val * 2


calc = BnPiCalculator()
calc.print_dAlem(10)
print("")
calc.print_ch(10)
print("")
calc.print_ch(140, 10)

d'Alembert
n = 1: 3.8729833462074166 (0 桁まで一致)
n = 2: 3.24037034920393 (0 桁まで一致)
n = 3: 3.1622776601683795 (1 桁まで一致)
n = 4: 3.146426544510455 (2 桁まで一致)
n = 5: 3.1427678676214668 (2 桁まで一致)
n = 6: 3.141882965902545 (3 桁まで一致)
n = 7: 3.1416648546853394 (3 桁まで一致)
n = 8: 3.1416106625961344 (3 桁まで一致)
n = 9: 3.1415971512924634 (5 桁まで一致)
n = 10: 3.141593777512209 (5 桁まで一致)

Cauchy-Hadamard
n = 1: 1.7320508075688772 (0 桁まで一致)
n = 2: 2.5900200641113513 (0 桁まで一致)
n = 3: 2.7908316238670965 (0 桁まで一致)
n = 4: 2.87938824021137 (0 桁まで一致)
n = 5: 2.9309182203146773 (0 桁まで一致)
n = 6: 2.965207805342988 (0 桁まで一致)
n = 7: 2.9898254214391717 (0 桁まで一致)
n = 8: 3.0083965582731698 (0 桁まで一致)
n = 9: 3.022914696021679 (0 桁まで一致)
n = 10: 3.034578330042229 (0 桁まで一致)

Cauchy-Hadamard
n = 1: 1.7320508075688772 (0 桁まで一致)
n = 11: 3.0441544716562277 (0 桁まで一致)
n = 21: 3.090170852802982 (0 桁まで一致)
n = 31: 3.1066658980313635 (1 桁まで一致)
n = 41: 3.1151486488948947 (1 桁まで一致)
n = 51: 3.1203161451311514 (1 桁まで一致)
n = 61: 3.1237941964255